<a href="https://colab.research.google.com/github/QuinAmii/File-Ingestion-and-Schema-Validation-Data-Glacier/blob/Main/File_Ingestion_and_Schema_Validation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np

In [30]:
data= pd.read_csv("/content/HR_Dataset.csv")
data

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2,151,3,0,1,0,support,low
14995,0.37,0.48,2,160,3,0,1,0,support,low
14996,0.37,0.53,2,143,3,0,1,0,support,low
14997,0.11,0.96,6,280,4,0,1,0,support,low


In [31]:
data.shape

(14999, 10)

In [8]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [12]:
# The size of the file
import os
print(os.path.getsize('/content/HR_Dataset.csv')/1024/1024/1024 , "GB")

0.000527859665453434 GB


In [85]:
# Reading csv without chunks.
import time
import pandas as pd
start = time.time()
data = pd.read_csv('/content/HR_Dataset.csv')
end = time.time()
print("Read csv without chunks: ",(end-start),"sec")

Read csv without chunks:  0.017284870147705078 sec


In [86]:
%%time
df.iloc[:,1].mean()

CPU times: user 494 µs, sys: 33 µs, total: 527 µs
Wall time: 537 µs


0.7161017401159978

In [87]:
# Reading csv with chunks.
start = time.time()
chunk = pd.read_csv('/content/HR_Dataset.csv',chunksize=1000000)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")
pd_data = pd.concat(chunk)

Read csv with chunks:  0.003622770309448242 sec


In [88]:
type(chunk)

pandas.io.parsers.TextFileReader

In [89]:
type(pd_data)

pandas.core.frame.DataFrame

In [90]:
%%time
pd_data.iloc[:,1].mean()

CPU times: user 513 µs, sys: 883 µs, total: 1.4 ms
Wall time: 1.06 ms


0.7161017401159978

In [92]:
pip install dask

In [94]:
pip install 'fsspec>=0.3.3'

     |████████████████████████████████| 132 kB 6.7 MB/s 


In [95]:
from dask import dataframe as dd
from dask.distributed import Client

In [97]:
pip install modin

     |████████████████████████████████| 761 kB 8.6 MB/s 
     |████████████████████████████████| 11.3 MB 45.6 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


In [4]:
from distributed import Client

client = Client()

Perhaps you already have a cluster running?
Hosting the HTTP server on port 38337 instead


In [2]:
 #almost 2secs for Dask
import dask.dataframe as dd
dt = dd.read_csv("/content/HR_Dataset.csv")

In [3]:
pip install modin[dask]

     |████████████████████████████████| 1.0 MB 8.9 MB/s 
     |████████████████████████████████| 802 kB 46.8 MB/s 
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Attempting uninstall: dask
    Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0
  Attempting uninstall: distributed
    Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.


In [8]:
import modin.pandas as pd
#almost 2secs too
data = pd.read_csv("/content/HR_Dataset.csv")

In [9]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [10]:
data.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'Department', 'salary'],
      dtype='object')

In [13]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [14]:
%%writefile file.yaml
file_type: csv
dataset_name: data
file_name: HR dataset
table_name: edsurv
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    -satisfaction_level
    -last_evaluation
    -number_project
    -average_montly_hours
    -time_spend_company
    -Work_accident
    -left
    -promotion_last_5years
    -Department
    -salary

Overwriting file.yaml


In [15]:
import testutility as util
config_data = util.read_config_file("file.yaml")

In [16]:
config_data['file_name']

'HR dataset'

In [17]:
#inspecting data of config file
config_data

{'columns': '-satisfaction_level -last_evaluation -number_project -average_montly_hours -time_spend_company -Work_accident -left -promotion_last_5years -Department -salary',
 'dataset_name': 'data',
 'file_name': 'HR dataset',
 'file_type': 'csv',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'edsurv'}

In [19]:
# read the file using config file
file_type = config_data['file_type']
source_file = "/content/" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv('/content/HR_Dataset.csv')
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [20]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['left', 'last_evaluation', 'salary', 'satisfaction_level', 'department', 'average_montly_hours', 'time_spend_company', 'work_accident', 'promotion_last_5years', 'number_project']
Following YAML columns are not in the file uploaded ['h', '-', 'g', 'b', 'p', 'u', 'n', ' ', 'f', 'e', 't', 's', 'j', 'v', 'l', 'i', 'a', 'k', 'd', 'r', 'm', 'y', '5', 'w', 'c', '_', 'o']


0

In [21]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'work_accident', 'left',
       'promotion_last_5years', 'department', 'salary'],
      dtype='object')
columns of YAML are: -satisfaction_level -last_evaluation -number_project -average_montly_hours -time_spend_company -Work_accident -left -promotion_last_5years -Department -salary


In [22]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['left', 'last_evaluation', 'salary', 'satisfaction_level', 'department', 'average_montly_hours', 'time_spend_company', 'work_accident', 'promotion_last_5years', 'number_project']
Following YAML columns are not in the file uploaded ['h', '-', 'g', 'b', 'p', 'u', 'n', ' ', 'f', 'e', 't', 's', 'j', 'v', 'l', 'i', 'a', 'k', 'd', 'r', 'm', 'y', '5', 'w', 'c', '_', 'o']
validation failed
